In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot, plot
init_notebook_mode(connected=True)

In [2]:
dataq = pd.read_csv('quake_db_1965-2016.csv')

In [3]:
country_code = 'JP'
country_df = dataq[dataq['Place'].str.contains(country_code)]
# country_df.head()

In [4]:
country_df = country_df[['Date', 'Latitude', 'Longitude', 'Magnitude', 'Depth', 'Place']]
_, _, country_df['Year'] = country_df['Date'].str.split('/').str
country_df.loc[:, 'Year'] = country_df.loc[:, 'Year'].astype(int)

In [5]:
country_df.head()

,Date,Latitude,Longitude,Magnitude,Depth,Place,Year
55,02/16/1965,38.908,142.095,5.7,53.5,"Ofunato, JP",1965
89,03/16/1965,40.697,143.032,6.4,32.1,"Kuji, JP",1965
101,03/29/1965,40.687,142.915,6.4,30.0,"Kuji, JP",1965
111,04/06/1965,36.083,139.968,5.7,50.0,"Ishige, JP",1965
122,04/15/1965,25.080,122.897,5.6,165.0,"Yonakuni, JP",1965


In [6]:
country_df.shape

(1347, 7)

In [7]:
years = list(set(country_df['Year'].to_list()))

In [8]:
def grabContentPerYear(year):
    cn_year_df = country_df[country_df['Year'] == int(year)]
    return cn_year_df

In [9]:
token = 'pk.eyJ1IjoiY2hhb3RpYy1lbmlnbWEiLCJhIjoiY2ppNXhxaWNiMG1qeTN4cjAwMTd0cnI1aCJ9.vPNGDu_1nXnuYAZ4pVZ9Fg'

In [13]:
frames = []
for year in years:
    cn_year_df = grabContentPerYear(year)
    lats = cn_year_df['Latitude'].to_list()
    lons = cn_year_df['Longitude'].to_list()
    frames.append(
        dict(data=[
            dict(
                type='scattermapbox',
                lat=lats,
                lon=lons,
                mode='markers',
                marker=dict(size=5, color=year)
            )
        ],
             traces=[0],
             name='frame{}'.format(year)
        )
    )

In [14]:
fig = go.Figure(
    data=[go.Scattermapbox(
        lat=[36.2048],
        lon=[138.2529],
        mode='markers',
        marker=dict(size=5)
    )],
    layout=go.Layout(width=800,
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            accesstoken=token,
            bearing=0,
            center=dict(
                lat=36.2048,
                lon=138.2529
            ),
            pitch=0,
            zoom=2,
            style='light'
        ),
        updatemenus=[
            dict(
                type="buttons",
                buttons=[dict(label="Play",
                          method="animate",
                          args=[None])]
            )
        ]
    ),
    frames=frames    
)

In [15]:
fig.show()